In [124]:
import tensorflow as tf
import random
import pandas as pd
import numpy as np

In [128]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

filename = "data/white.csv"

# setup text reader
file_length = file_len(filename)
filename_queue = tf.train.string_input_producer([filename])
reader = tf.TextLineReader(skip_header_lines=1)
_, csv_row = reader.read(filename_queue)

# setup CSV decoding
record_defaults = [[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0]]
x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,label= tf.decode_csv(csv_row, record_defaults=record_defaults)
label = tf.one_hot(label,depth=7)

features = tf.stack([x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11])

In [129]:
tf.reset_default_graph()

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

filename = "data/white.csv"

# setup text reader
file_length = file_len(filename)
filename_queue = tf.train.string_input_producer([filename])
reader = tf.TextLineReader(skip_header_lines=1)
_, csv_row = reader.read(filename_queue, )

# setup CSV decoding
record_defaults = [[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0]]
x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,label= tf.decode_csv(csv_row, record_defaults=record_defaults)
label = tf.one_hot(label,depth=7)


features = tf.stack([x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11])

X = tf.placeholder(tf.float32, [None, 11]) # 학습데이터는 11개의 features
Y = tf.placeholder(tf.float32, [None, 7]) # 라벨 데이터는 7가지의 label

keep_prob = tf.placeholder(tf.float32) # dropout rate

W1 = tf.get_variable("W1", shape=[11, 44], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([44])) # bias는 가중치의 열의 개수에 맞춘다.
L1 = tf.nn.relu(tf.matmul(X,W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[44,22], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([22])) # bias는 가중치의 열의 개수에 맞춘다.
L2 = tf.nn.relu(tf.matmul(L1,W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[22,7], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([7])) # bias는 가중치의 열의 개수에 맞춘다.
L3 = tf.nn.relu(tf.matmul(L2,W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("W4", shape=[7,2], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([2])) # bias는 가중치의 열의 개수에 맞춘다.

regression = tf.matmul(L2, W3) + b3 # 최종 그래프

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=regression, labels=Y))

#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam 짱....


with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    with tf.Graph().as_default():
      # start populating filename queue
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)

        for i in range(500):
        # retrieve a single instance
            for j in range(file_length):
                example,labels = sess.run([features,label])
                feed_dict = {X:example, Y:labels, keep_prob: 0.7}
                c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
                avg = avg + c
            avg = avg/file_length
            print('cost =', '{:.4f}'.format(avg))
        coord.request_stop()
        coord.join(threads)

with tf.Session() as sess:
    tf.initialize_all_variables().run()

  # start populating filename queue
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    for i in range(10):
    # retrieve a single instance
        example,labels = sess.run([features,label])
        print(example)

    coord.request_stop()
    coord.join(threads)
sess.close()

[7.000e+00 2.700e-01 3.600e-01 2.070e+01 4.500e-02 4.500e+01 1.700e+02
 1.001e+00 3.000e+00 4.500e-01 8.800e+00]
[6.30e+00 3.00e-01 3.40e-01 1.60e+00 4.90e-02 1.40e+01 1.32e+02 9.94e-01
 3.30e+00 4.90e-01 9.50e+00]
[8.100e+00 2.800e-01 4.000e-01 6.900e+00 5.000e-02 3.000e+01 9.700e+01
 9.951e-01 3.260e+00 4.400e-01 1.010e+01]
[7.200e+00 2.300e-01 3.200e-01 8.500e+00 5.800e-02 4.700e+01 1.860e+02
 9.956e-01 3.190e+00 4.000e-01 9.900e+00]
[7.200e+00 2.300e-01 3.200e-01 8.500e+00 5.800e-02 4.700e+01 1.860e+02
 9.956e-01 3.190e+00 4.000e-01 9.900e+00]
[8.100e+00 2.800e-01 4.000e-01 6.900e+00 5.000e-02 3.000e+01 9.700e+01
 9.951e-01 3.260e+00 4.400e-01 1.010e+01]
[6.200e+00 3.200e-01 1.600e-01 7.000e+00 4.500e-02 3.000e+01 1.360e+02
 9.949e-01 3.180e+00 4.700e-01 9.600e+00]
[7.000e+00 2.700e-01 3.600e-01 2.070e+01 4.500e-02 4.500e+01 1.700e+02
 1.001e+00 3.000e+00 4.500e-01 8.800e+00]
[6.30e+00 3.00e-01 3.40e-01 1.60e+00 4.90e-02 1.40e+01 1.32e+02 9.94e-01
 3.30e+00 4.90e-01 9.50e+00]
[8.10

In [ ]:
X = tf.placeholder(tf.float32, [None, 11]) # 학습데이터는 11개의 features
Y = tf.placeholder(tf.float32, [None, 7]) # 라벨 데이터는 7가지의 label

keep_prob = tf.placeholder(tf.float32) # dropout rate

W1 = tf.get_variable("W1", shape=[11, 44], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([44])) # bias는 가중치의 열의 개수에 맞춘다.
L1 = tf.nn.relu(tf.matmul(X,W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[44,22], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([22])) # bias는 가중치의 열의 개수에 맞춘다.
L2 = tf.nn.relu(tf.matmul(L1,W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[22,7], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([7])) # bias는 가중치의 열의 개수에 맞춘다.

regression = tf.matmul(L2, W3) + b3 # 최종 그래프

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=regression, labels=Y))

#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam 짱....


with tf.Session() as sess:
    tf.initialize_all_variables().run()

  # start populating filename queue
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    for i in range(500):
    # retrieve a single instance
        for j in range(file_length)
            example,labels = sess.run([features,label])
            feed_dict = {X:example, Y:labels, keep_prob: 0.7}
            c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            avg = avg + c
        avg = avg/file_length
        print('cost =', '{:.4f}'.format(avg))
    coord.request_stop()
    coord.join(threads)